In [1]:
import pandas as pd

In [3]:
actors_df = pd.read_pickle('actorCast.pkl')
actors_df

,tconst,actorName_x,actorName_y,actorName
0,tt0025164,Fred Astaire,Ginger Rogers,Alice Brady
1,tt0026942,Irene Dunne,Fred Astaire,Ginger Rogers
2,tt0027125,Fred Astaire,Ginger Rogers,Edward Everett Horton
3,tt0027630,Fred Astaire,Ginger Rogers,Randolph Scott
4,tt0028333,Fred Astaire,Ginger Rogers,Victor Moore
...,...,...,...,...
3974870,tt8743674,Beverly,NaN,NaN
3974871,tt8743704,Jessy,NaN,NaN
3974872,tt8743714,Nyla,NaN,NaN
3974873,tt8744122,Yukari Funaki,Hiroko Ide,NaN


In [4]:
director_df = pd.read_pickle('directors.pkl')
director_df

,tconst,directorName
0,tt0038468,Ingmar Bergman
1,tt0038675,Ingmar Bergman
2,tt0039834,Ingmar Bergman
3,tt0040418,Ingmar Bergman
4,tt0040622,Ingmar Bergman
...,...,...
3456862,tt8743742,Fotis Georgopoulos
3456863,tt8744074,Art Jones
3456864,tt8744160,Ibrahim-Aloduley
3456865,tt9046122,Eli Bevins


In [5]:
movies_df = pd.read_pickle('moviesData.pkl')
movies_df

,tconst,title,year,genres
15492,tt0015724,Dama de noche,1993,"Drama,Mystery,Romance"
16662,tt0016906,Frivolinas,2014,"Comedy,Musical"
34811,tt0035423,Kate & Leopold,2001,"Comedy,Fantasy,Romance"
35975,tt0036606,"Another Time, Another Place",1983,"Drama,War"
38019,tt0038687,Let There Be Light,1980,"Documentary,War"
...,...,...,...,...
7294790,tt9916538,Kuambil Lagi Hatiku,2019,Drama
7294831,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,Documentary
7294858,tt9916680,De la ilusión al desconcierto: cine colombiano...,2007,Documentary
7294870,tt9916706,Dankyavar Danka,2013,Comedy


In [6]:
ratings_df = pd.read_pickle('ratingsData.pkl')
ratings_df

,tconst,averageRating
0,tt0000001,5.6
1,tt0000002,6.1
2,tt0000003,6.5
3,tt0000004,6.2
4,tt0000005,6.2
...,...,...
1085490,tt9916580,7.2
1085491,tt9916690,6.6
1085492,tt9916720,6.0
1085493,tt9916766,6.9


In [27]:
# Rename columns
actors_df = actors_df.rename(columns={'actorName_x':'actor_1', 'actorName_y':'actor_2','actorName':'actor_3'})
director_df = director_df.rename(columns={'directorName':'director'})
ratings_df = ratings_df.rename(columns={'averageRating':'rating'})

In [28]:
# Fill null values in actors 2 and actors 3
actors_df[['actor_2','actor_3']]=actors_df[['actor_2','actor_3']].fillna('')

In [29]:
# Remove comma from genres
movies_df['genres'] = movies_df['genres'].str.replace(',', ' ')
movies_df

,tconst,title,year,genres
15492,tt0015724,Dama de noche,1993,Drama Mystery Romance
16662,tt0016906,Frivolinas,2014,Comedy Musical
34811,tt0035423,Kate & Leopold,2001,Comedy Fantasy Romance
35975,tt0036606,"Another Time, Another Place",1983,Drama War
38019,tt0038687,Let There Be Light,1980,Documentary War
...,...,...,...,...
7294790,tt9916538,Kuambil Lagi Hatiku,2019,Drama
7294831,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,Documentary
7294858,tt9916680,De la ilusión al desconcierto: cine colombiano...,2007,Documentary
7294870,tt9916706,Dankyavar Danka,2013,Comedy


### Join all dataframes in one

In [30]:
final_movies_df= pd.merge(movies_df,actors_df,on='tconst',how='inner').merge(director_df,on='tconst',how='inner').merge(ratings_df,on='tconst',how='left')

In [31]:
final_movies_df

,tconst,title,year,genres,actor_1,actor_2,actor_3,director,rating
0,tt0015724,Dama de noche,1993,Drama Mystery Romance,Rafael Sánchez Navarro,Cecilia Toussaint,Miguel Córcega,Eva López Sánchez,6.2
1,tt0016906,Frivolinas,2014,Comedy Musical,José López Alonso,Juan Belmonte,María Caballé,Arturo Carballo,5.6
2,tt0035423,Kate & Leopold,2001,Comedy Fantasy Romance,Meg Ryan,Hugh Jackman,Liev Schreiber,James Mangold,6.4
3,tt0036606,"Another Time, Another Place",1983,Drama War,Phyllis Logan,Giovanni Mauriello,Gianluca Favilla,Michael Radford,6.5
4,tt0038687,Let There Be Light,1980,Documentary War,Walter Huston,,,John Huston,7.4
...,...,...,...,...,...,...,...,...,...
207475,tt9916186,Illenau - die Geschichte einer ehemaligen Heil...,2017,Documentary,David Nathan,Helmut Schiffner,Andree Steinke,Frank König,NaN
207476,tt9916362,Coven,2020,Drama History Horror,Amaia Aberasturi,Alex Brendemühl,Daniel Fanego,Pablo Agüero,6.1
207477,tt9916428,The Secret of China,2019,Adventure History War,Kenan Heppe,Wang Peng Kai,Valery Gadreau,Jixing Wang,4.1
207478,tt9916538,Kuambil Lagi Hatiku,2019,Drama,Lala Karmela,Cut Mini Theo,Sahil Shah,Azhar Kinoi Lubis,8.4


In [32]:
# added new column
final_movies_df['combined']= final_movies_df['genres']+" "+final_movies_df['actor_1']+" "+final_movies_df['actor_2']+" "+final_movies_df['actor_3']+" "+final_movies_df['director']

In [33]:
final_movies_df

,tconst,title,year,genres,actor_1,actor_2,actor_3,director,rating,combined
0,tt0015724,Dama de noche,1993,Drama Mystery Romance,Rafael Sánchez Navarro,Cecilia Toussaint,Miguel Córcega,Eva López Sánchez,6.2,Drama Mystery Romance Rafael Sánchez Navarro C...
1,tt0016906,Frivolinas,2014,Comedy Musical,José López Alonso,Juan Belmonte,María Caballé,Arturo Carballo,5.6,Comedy Musical José López Alonso Juan Belmonte...
2,tt0035423,Kate & Leopold,2001,Comedy Fantasy Romance,Meg Ryan,Hugh Jackman,Liev Schreiber,James Mangold,6.4,Comedy Fantasy Romance Meg Ryan Hugh Jackman L...
3,tt0036606,"Another Time, Another Place",1983,Drama War,Phyllis Logan,Giovanni Mauriello,Gianluca Favilla,Michael Radford,6.5,Drama War Phyllis Logan Giovanni Mauriello Gia...
4,tt0038687,Let There Be Light,1980,Documentary War,Walter Huston,,,John Huston,7.4,Documentary War Walter Huston John Huston
...,...,...,...,...,...,...,...,...,...,...
207475,tt9916186,Illenau - die Geschichte einer ehemaligen Heil...,2017,Documentary,David Nathan,Helmut Schiffner,Andree Steinke,Frank König,NaN,Documentary David Nathan Helmut Schiffner Andr...
207476,tt9916362,Coven,2020,Drama History Horror,Amaia Aberasturi,Alex Brendemühl,Daniel Fanego,Pablo Agüero,6.1,Drama History Horror Amaia Aberasturi Alex Bre...
207477,tt9916428,The Secret of China,2019,Adventure History War,Kenan Heppe,Wang Peng Kai,Valery Gadreau,Jixing Wang,4.1,Adventure History War Kenan Heppe Wang Peng Ka...
207478,tt9916538,Kuambil Lagi Hatiku,2019,Drama,Lala Karmela,Cut Mini Theo,Sahil Shah,Azhar Kinoi Lubis,8.4,Drama Lala Karmela Cut Mini Theo Sahil Shah Az...


In [36]:
final_movies_df.to_pickle("Final_MovieData.pkl")